# Import CSV into PostgreSQL database
The goal of this project is to import a CSV file into a PostgreSQL database.It's important that you already have a PostgreSQL server installed on your machine and have created user access for the database. The steps to be followed are:
1. Understand the data from the CSV file
2. Create a SQL table that fits the data
3. Import a single value to the database
4. Import a row to the database
5. Import all the rows
Let's see which surprises await!

# Step 1: read csv
Firstly, we need to be able to load the data into Python and get a feel for what it looks like. We'll use the csv library in Python to handle the file import. Let's print the column headers and first row of data to get a better feel for what the data looks like.

In [7]:
import csv
with open('austin_weather.csv', 'r') as f:
    reader = csv.reader(f)
    columns = next(reader)
    first_row = next(reader)
    second_row = next(reader)
    print(columns)
    print(first_row)


['Date', 'TempHighF', 'TempAvgF', 'TempLowF', 'DewPointHighF', 'DewPointAvgF', 'DewPointLowF', 'HumidityHighPercent', 'HumidityAvgPercent', 'HumidityLowPercent', 'SeaLevelPressureHighInches', 'SeaLevelPressureAvgInches', 'SeaLevelPressureLowInches', 'VisibilityHighMiles', 'VisibilityAvgMiles', 'VisibilityLowMiles', 'WindHighMPH', 'WindAvgMPH', 'WindGustMPH', 'PrecipitationSumInches', 'Events']
['2013-12-21', '74', '60', '45', '67', '49', '43', '93', '75', '57', '29.86', '29.68', '29.59', '10', '7', '2', '20', '4', '31', '0.46', 'Rain , Thunderstorm']


We can see that the first row contains the headers, after that the values follow. With the exception of the first and the last columns, data seems to be numerical. Let's make sure that all the rows within the file contain the same amount of columns.

In [8]:
with open('austin_weather.csv', 'r') as f:
    reader = csv.reader(f)
    f.seek(0)
    for row in f:
        my_row = next(reader)
        if not len(my_row) == 21:
            print ("row length not 21!")

Good news everyone! All rows contain 21 values, so no need to worry about the shape of the file. Next, let's create an SQL table with a column for each value.

# Step 2: create an SQL table which fits the data
The statement sql_create_weather_table will create the new table for us. Each column in our .csv file gets its own column in the table. Note that an ID column was added (which is not contained in the orginial data file). This is because the SQL database requires a unique field. We've set the ID to "serial" so that it will automatically increase when a new value is added.

In [11]:
import psycopg2
sql_create_weather_table = """CREATE TABLE IF NOT EXISTS weather(
                                ID SERIAL PRIMARY KEY,
                                date date,
                                TempHighF integer,
                                TempAvgF integer,
                                TempLowF integer,
                                DewPointHighF integer,
                                DewPointAvgF integer,
                                DewPointLowF integer,
                                HumidityHighPercent integer,
                                HumidityAvgPercent integer,
                                HumidityLowPercent integer,
                                SeaLevelPressureHighInches real,
                                SeaLevelPressureAvgInches real,
                                SeaLevelPressureLowInches real,
                                VisibilityHighMiles integer,
                                VisibilityAvgMiles integer,
                                VisibilityLowMiles integer,
                                WindHighMPH integer,
                                WindAvgMPH integer,
                                WindGustMPH integer,
                                PrecipitationSumInches real,
                                Events text
                                )"""   

# create a connection with the database (this should already exist)
try:
    connection = psycopg2.connect("dbname='austin_weather_energy' user='muriel' host='localhost' password='1'")
    print("connected to austin_weather_energy")
except:
    print("Unable to connect to the database")
# the cursor can help us execute SQL
cursor = connection.cursor()

# delete the table if it already exists
sql = """DROP table weather"""
cursor.execute(sql)
connection.commit()

# now let's create the table
cursor.execute(sql_create_weather_table)
# and commit to the DB
connection.commit()

# next, let's print the column names to see if it worked:


def print_values():
    cursor.execute("SELECT * from weather")
    colnames = [desc[0] for desc in cursor.description]
    print("Columns in database:")
    print(colnames)
    rows = cursor.fetchall()
    print("Values in database:")
    for row in rows[0:4]: #only print first 5 rows to avoid clutter
        print(" ", row)
    connection.commit()

# print the first 5 values     
print_values()

connected to austin_weather_energy
Columns in database:
['id', 'date', 'temphighf', 'tempavgf', 'templowf', 'dewpointhighf', 'dewpointavgf', 'dewpointlowf', 'humidityhighpercent', 'humidityavgpercent', 'humiditylowpercent', 'sealevelpressurehighinches', 'sealevelpressureavginches', 'sealevelpressurelowinches', 'visibilityhighmiles', 'visibilityavgmiles', 'visibilitylowmiles', 'windhighmph', 'windavgmph', 'windgustmph', 'precipitationsuminches', 'events']
Values in database:


Hurrah! We've managed to create a table in our database. At this point in time the table is empty (as shown by the print_values function. Next, let's add some values:

# Step 3: Import a single value into the Database
I always like breaking complex problems into little pieces, so let's get started by importing only a single value into the database and print that. If that works, we can move on to more complicated stuff.

In [12]:
# now let's try to add a single value to the database
print(first_row[0])
sql = """INSERT INTO weather(date) VALUES (%r)""" %(first_row[0])
cursor.execute(sql)
connection.commit()
print_values()

2013-12-21
Columns in database:
['id', 'date', 'temphighf', 'tempavgf', 'templowf', 'dewpointhighf', 'dewpointavgf', 'dewpointlowf', 'humidityhighpercent', 'humidityavgpercent', 'humiditylowpercent', 'sealevelpressurehighinches', 'sealevelpressureavginches', 'sealevelpressurelowinches', 'visibilityhighmiles', 'visibilityavgmiles', 'visibilitylowmiles', 'windhighmph', 'windavgmph', 'windgustmph', 'precipitationsuminches', 'events']
Values in database:
  (1, datetime.date(2013, 12, 21), None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)


We've sucessfully managed to input the first value (which happens to be a date) into the first row. Note the format of the date (datetime.date) which means that Postgres automatically converted the string from the CSV into a datetime. Here we need to be careful, dates can go wront in many different ways - but let's not think about that for the time being.

Everything else is still empty (or "None" in PostgreSQL terms), but this proves that we are able to write to the database and read & display values. Good progress!

# Step 4: Import a row
With data you never know where it's going to go wrong, points might ben decimals or stings can contain characters which break everything. If we try to dump the entire csv to the db, we won't know where things went wrong, so let's try to input each column at least once to ensure there are no conflicts.

In [13]:
# next let's try to add the first row's values into the database one by one. This will help us
# understand if there are any conflicts between the data types of the csv versus the SQL database

for colname, value in zip(columns, first_row):
    sql = """INSERT INTO weather(%s) VALUES (%r)""" %(colname,value)
    #print(sql)
    cursor.execute(sql)
    connection.commit()
# print the first 5 values    
print_values()    
# note that each item is inserted in a new row, which is not what we want, but for now it's good
# enough as we're only trying to check if the data can be transferred.

Columns in database:
['id', 'date', 'temphighf', 'tempavgf', 'templowf', 'dewpointhighf', 'dewpointavgf', 'dewpointlowf', 'humidityhighpercent', 'humidityavgpercent', 'humiditylowpercent', 'sealevelpressurehighinches', 'sealevelpressureavginches', 'sealevelpressurelowinches', 'visibilityhighmiles', 'visibilityavgmiles', 'visibilitylowmiles', 'windhighmph', 'windavgmph', 'windgustmph', 'precipitationsuminches', 'events']
Values in database:
  (1, datetime.date(2013, 12, 21), None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
  (2, datetime.date(2013, 12, 21), None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
  (3, None, 74, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
  (4, None, None, 60, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None

No errors, that's great news! We've also managed to print the first five rows to see that the auto-incriment function works. Each value is in it's own new row at the moment, because the database creates a new row each tie the cursor executes. This is not what we want,so let's clean that up before continuing. Notice the frightening eas with which SQL deletes an entire table, no questions asked... Yikes!

In [15]:
# delete the table if it already exists
sql = """DROP table weather"""
cursor.execute(sql)
connection.commit()

# now let's create the table
cursor.execute(sql_create_weather_table)
# and commit to the DB
connection.commit()

# Step 5: Import all the rows
When we try to import all the rows, we get SQL errors a bunch of times. Trial and error shows that the data needs to be pre-processed for the following:
1. adding quotation marks around the "Date" column (otherwise the "-" is interpreted as a minus)
2. adding quotation marks around the "Events" column (some of the values contain commas)
3. Replacing "-" and "F" numerical values with NULL

In [16]:
with open('austin_weather.csv', 'r') as f:
    reader = csv.reader(f)
    columns = next(reader)  
    f.seek(0)
    for row in f:
        my_row = next(reader)
        last_value = my_row[len(my_row)-1]
        # Add ' around the text in the last column to prevent and SQL error for handling a string with a comma:
        last_value = "'"+last_value+"'"    
        my_row[len(my_row)-1] = last_value
        my_row[0] = "'"+my_row[0]+"'"
        # Replace non-numerical values with NULL
        for j in range(0,len(my_row)):
            if my_row[j] == "T":
                my_row[j] = "NULL"
            if my_row[j] == "-":
                my_row[j] = "NULL"
        sql = """INSERT INTO weather({0}) VALUES ({1})"""
        sql = sql.format(','.join(columns), ','.join(my_row))
        cursor.execute(sql)
connection.commit()

This works without errors! Next, let's see the data in our database (we'll only pring the first five rows to avoid cluttering):

In [17]:
print_values()

Columns in database:
['id', 'date', 'temphighf', 'tempavgf', 'templowf', 'dewpointhighf', 'dewpointavgf', 'dewpointlowf', 'humidityhighpercent', 'humidityavgpercent', 'humiditylowpercent', 'sealevelpressurehighinches', 'sealevelpressureavginches', 'sealevelpressurelowinches', 'visibilityhighmiles', 'visibilityavgmiles', 'visibilitylowmiles', 'windhighmph', 'windavgmph', 'windgustmph', 'precipitationsuminches', 'events']
Values in database:
  (1, datetime.date(2013, 12, 21), 74, 60, 45, 67, 49, 43, 93, 75, 57, 29.86, 29.68, 29.59, 10, 7, 2, 20, 4, 31, 0.46, 'Rain , Thunderstorm')
  (2, datetime.date(2013, 12, 23), 58, 45, 32, 31, 27, 23, 76, 52, 27, 30.56, 30.49, 30.41, 10, 10, 10, 8, 3, 12, 0.0, ' ')
  (3, datetime.date(2013, 12, 25), 58, 50, 41, 44, 40, 36, 86, 71, 56, 30.41, 30.33, 30.27, 10, 10, 7, 10, 2, 16, None, ' ')
  (4, datetime.date(2013, 12, 27), 60, 53, 45, 41, 39, 37, 83, 65, 47, 30.46, 30.39, 30.34, 10, 9, 7, 7, 1, 11, None, ' ')


In [66]:
# and finally let's close the connection and the cursor:
connection.close()
cursor.close()

# Conclusion
It's possible to import a .csv file into a PostGreSQL database using the psycopg2 and csv libraries in Python. It's a good idea to first understand the data in the csv and start small, first importing only one value, then values one at a time and finally one row at a time. A very important part of the process is checking the data every step of the way and making sure what you thought would happen actually happened.